<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4f2c7f76f2a2d89547a47c00352a2384a83005a1fd077499b843990d091984f7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-10 12:23:58
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 26.08 K (0.19%)
Current PnL: -28.86 L (-18.64%)
CY Booked + Current PnL: -14.57 L (-9.41%)
-------------------
Total profit:  1.22 L
Total loss:  -30.08 L
-------------------
Total Booked + Current PnL: 12.41 L (9.71%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.25%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.49 L (67.06%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,1.10,-17.12,20.77,0.09,17347.0,-17258.0,83519.0,93.81,39.0,M-SC,2.35,253.0,-0.99,0.62,9.73,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,2.81,-12.29,14.04,0.02,21496.0,-21456.0,153103.0,133.26,53.0,M-SC,16.64,234.0,-1.00,1.14,33.17,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.66,-2.30,24.92,22.05,23856.0,-2250.0,95730.0,-15.99,55.0,H-SC,6.86,164.0,-0.09,0.71,39.15,XR,ATH,FINANCE
43,ITC,452.00,-0.31,-2.01,12.70,10.44,24906.0,-4025.0,196113.0,-41.56,43.0,X-LC,1.67,5.0,-0.16,1.46,3.44,X40,NTT,FMCG
82,VOLTAS,1530.00,0.31,4.84,14.17,19.69,28484.0,9273.0,201015.0,-5.36,45.0,X-MC,2.57,78.0,0.33,1.49,11.96,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.0,4.98,-34.81,53.37,-0.01,44469.0,-44483.0,83322.0,16943.48,29.0,M-SC,4.08,233.0,-1.00,0.62,6.64,OX40N,NTT,AC
18,COFFEEDAY,80.0,4.57,-43.86,140.96,35.27,89854.0,-49805.0,63744.0,-57.75,33.0,L-SC,10.16,270.0,-0.55,0.47,56.02,XR,NTT,HOTELS
36,ICICIPRULI,790.0,2.92,6.40,23.09,30.97,43420.0,11309.0,188047.0,-18.62,64.0,X-MC,1.96,75.0,0.26,1.40,19.76,X40,ATH,INSURANCE
39,INDIGOPNTS,1408.0,2.81,-12.29,14.04,0.02,21496.0,-21456.0,153103.0,133.26,53.0,M-SC,16.64,234.0,-1.00,1.14,33.17,OX40N,NTT,PAINTS
62,SATIN,274.0,2.73,-21.87,94.33,51.83,203631.0,-60413.0,215871.0,-30.35,36.0,H-SC,1.95,146.0,-0.30,1.61,5.16,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DMART,5391.45,-2.54,-9.45,41.49,28.12,79049.0,-19882.0,190525.0,-21.92,29.0,X-LC,4.64,19.0,-0.25,1.42,11.94,X40N,ATH,FMCG
61,ROUTE,2227.21,-2.34,-48.74,229.76,69.05,156731.0,-64851.0,68215.0,-58.67,46.0,H-SC,23.20,140.0,-0.41,0.51,5.20,SR,ATH,IT
57,RAJOOENG,143.10,-2.30,-37.09,121.93,39.61,78620.0,-38020.0,64480.0,-57.37,23.0,H-SC,26.25,135.0,-0.48,0.48,0.00,AR,ATH,MISC
74,TITAGARH,1548.00,-2.02,-28.62,104.17,45.74,180079.0,-69307.0,172870.0,-4.90,26.0,H-SC,4.03,174.0,-0.38,1.29,12.43,XY24,NTT,ENGINEERING
59,RELAXO,1176.00,-1.76,-46.45,184.40,52.31,143349.0,-67422.0,77738.0,-42.99,48.0,X-SC,6.14,91.0,-0.47,0.58,4.06,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.63,-6.49,122.34,107.91,168905.0,-9582.0,138062.0,-23.79,39.0,M-SC,12.16,216.0,-0.06,1.03,2.94,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.81,-12.29,14.04,0.02,21496.0,-21456.0,153103.0,133.26,53.0,M-SC,16.64,234.0,-1.00,1.14,33.17,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.10,-17.12,20.77,0.09,17347.0,-17258.0,83519.0,93.81,39.0,M-SC,2.35,253.0,-0.99,0.62,9.73,OX40N,NTT,DURABLES
66,SIS,528.00,0.21,-24.50,60.61,21.26,50810.0,-27201.0,83831.0,1977.20,47.0,H-SC,4.70,166.0,-0.54,0.62,13.26,OX40N,NTT,MISC
47,KANSAINER,340.00,0.94,-25.24,51.79,13.47,104409.0,-68067.0,201600.0,-69.28,30.0,H-SC,3.66,158.0,-0.65,1.50,2.51,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.68,6.3,62.06,72.27,111780.0,10668.0,180116.0,-8.8,66.0,M-LC,2.08,99.0,0.1,1.34,12.86,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.68,6.30,62.06,72.27,111780.0,10668.0,180116.0,-8.80,66.0,M-LC,2.08,99.0,0.10,1.34,12.86,XR,NTT,IT
38,INDIAMART,4810.62,-0.84,-3.79,114.87,106.72,136300.0,-4680.0,118656.0,-52.34,31.0,H-SC,1.58,139.0,-0.03,0.88,18.70,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.91,-2.07,38.50,35.63,79014.0,-4342.0,205232.0,-15.85,41.0,H-MC,3.15,119.0,-0.05,1.53,14.25,AR,ATH,PHARMA
25,FINCABLES,1641.55,1.63,-6.49,122.34,107.91,168905.0,-9582.0,138062.0,-23.79,39.0,M-SC,12.16,216.0,-0.06,1.03,2.94,OX40N,ATH,CABLES
37,IEX,219.00,-0.47,-4.64,55.85,48.63,106498.0,-9268.0,190686.0,-36.48,46.0,H-SC,14.08,136.0,-0.09,1.42,6.38,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-1.12,-14.16,41.64,21.58,50090.0,-19845.0,120294.0,-56.73,23.0,X-MC,6.06,56.0,-0.40,0.89,6.50,X40N,ATH,IT
9,BAJAJHFL,181.50,0.65,-21.36,88.77,48.45,139978.0,-42820.0,157686.0,-26.96,25.0,X-MC,11.68,64.0,-0.31,1.17,2.67,X40N,ATH,FINANCE
75,TMPV,600.00,-0.29,-18.55,74.57,42.18,167875.0,-51286.0,225124.0,-26.74,25.0,X-LC,5.62,3.0,-0.31,1.67,0.00,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,-0.28,-10.52,27.04,13.67,62195.0,-27053.0,230010.0,-17.28,28.0,X-LC,2.79,14.0,-0.43,1.71,8.58,XY25,NTT,FMCG
53,PAGEIND,51605.07,-0.63,-9.71,39.62,26.07,58574.0,-15900.0,147840.0,-33.51,29.0,X-MC,11.86,55.0,-0.27,1.10,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,0.00,-22.13,97.11,53.50,228187.0,-66773.0,234978.0,-62.35,31.0,X-MC,1.35,58.0,-0.29,1.75,4.37,XY24,NTT,FMCG
15,CAMS,4762.0,-0.97,-80.55,541.61,24.79,265308.0,-202871.0,48985.0,-81.54,38.0,X-SC,1.52,86.0,-0.76,0.36,19.75,X40N,NTT,MISC
43,ITC,452.0,-0.31,-2.01,12.70,10.44,24906.0,-4025.0,196113.0,-41.56,43.0,X-LC,1.67,5.0,-0.16,1.46,3.44,X40,NTT,FMCG
36,ICICIPRULI,790.0,2.92,6.40,23.09,30.97,43420.0,11309.0,188047.0,-18.62,64.0,X-MC,1.96,75.0,0.26,1.40,19.76,X40,ATH,INSURANCE
3,ACC,3906.0,-0.01,-24.99,118.85,64.15,212123.0,-59471.0,178480.0,-56.31,35.0,X-SC,1.97,82.0,-0.28,1.33,0.12,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.63,-9.71,39.62,26.07,58574.0,-15900.0,147840.0,-33.51,29.0,X-MC,11.86,55.0,-0.27,1.10,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.29,-18.55,74.57,42.18,167875.0,-51286.0,225124.0,-26.74,25.0,X-LC,5.62,3.0,-0.31,1.67,0.00,XY24,NTT,AUTO
54,PGHH,17907.65,0.02,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,34.0,X-MC,5.15,57.0,-0.15,1.41,0.10,X40,ATH,FMCG
3,ACC,3906.00,-0.01,-24.99,118.85,64.15,212123.0,-59471.0,178480.0,-56.31,35.0,X-SC,1.97,82.0,-0.28,1.33,0.12,XY24,BTT,CEMENT
31,HAVELLS,2069.16,-0.46,-9.52,46.42,32.48,137762.0,-31219.0,296772.0,-14.05,39.0,X-MC,5.26,67.0,-0.23,2.21,0.82,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.32,-30.19,104.60,42.84,47471.0,-19623.0,45383.0,-54.27,40.0,X-SC,36.83,83.0,-0.41,0.34,2.68,XY24,NTT,MISC
15,CAMS,4762.00,-0.97,-80.55,541.61,24.79,265308.0,-202871.0,48985.0,-81.54,38.0,X-SC,1.52,86.0,-0.76,0.36,19.75,X40N,NTT,MISC
59,RELAXO,1176.00,-1.76,-46.45,184.40,52.31,143349.0,-67422.0,77738.0,-42.99,48.0,X-SC,6.14,91.0,-0.47,0.58,4.06,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.26,-36.35,112.40,35.20,92060.0,-46766.0,81904.0,7.44,40.0,X-SC,15.39,92.0,-0.51,0.61,4.04,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.12,-14.16,41.64,21.58,50090.0,-19845.0,120294.0,-56.73,23.0,X-MC,6.06,56.0,-0.40,0.89,6.50,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.55,-13.31,37.59,19.28,112742.0,-46030.0,299926.0,-24.27,59.0,X-LC,5.49,1.0,-0.41,2.23,10.88,X40,ATH,IT
41,INFY,2275.00,-0.33,8.49,42.75,54.87,145118.0,26570.0,339458.0,-15.65,62.0,X-LC,3.30,2.0,0.18,2.52,17.48,X40,BTT,IT
75,TMPV,600.00,-0.29,-18.55,74.57,42.18,167875.0,-51286.0,225124.0,-26.74,25.0,X-LC,5.62,3.0,-0.31,1.67,0.00,XY24,NTT,AUTO
81,VBL,671.64,0.03,-4.72,42.39,35.67,127571.0,-14897.0,300945.0,-16.08,53.0,X-LC,4.73,4.0,-0.12,2.24,8.48,X40N,ATH,FMCG
43,ITC,452.00,-0.31,-2.01,12.70,10.44,24906.0,-4025.0,196113.0,-41.56,43.0,X-LC,1.67,5.0,-0.16,1.46,3.44,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.46,-17.57,120.40,81.67,93102.0,-16483.0,77327.0,6950.00,41.0,L-SC,14.31,271.0,-0.18,0.58,49.75,XR,NTT,CERAMICS
50,MASFIN,398.61,0.66,-2.30,24.92,22.05,23856.0,-2250.0,95730.0,-15.99,55.0,H-SC,6.86,164.0,-0.09,0.71,39.15,XR,ATH,FINANCE
13,BSOFT,831.70,-1.07,-22.80,95.33,50.79,103508.0,-32074.0,108579.0,-1.04,66.0,H-SC,5.83,151.0,-0.31,0.81,26.42,XR,ATH,IT
39,INDIGOPNTS,1408.00,2.81,-12.29,14.04,0.02,21496.0,-21456.0,153103.0,133.26,53.0,M-SC,16.64,234.0,-1.00,1.14,33.17,OX40N,NTT,PAINTS
36,ICICIPRULI,790.00,2.92,6.40,23.09,30.97,43420.0,11309.0,188047.0,-18.62,64.0,X-MC,1.96,75.0,0.26,1.40,19.76,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.07,-22.80,95.33,50.79,103508.0,-32074.0,108579.0,-1.04,66.0,H-SC,5.83,151.0,-0.31,0.81,26.42,XR,ATH,IT
84,WIPRO,420.00,0.68,6.30,62.06,72.27,111780.0,10668.0,180116.0,-8.80,66.0,M-LC,2.08,99.0,0.10,1.34,12.86,XR,NTT,IT
32,HCLTECH,1922.01,0.33,7.92,15.57,24.73,40652.0,19167.0,261091.0,12.76,63.0,X-LC,5.97,7.0,0.47,1.94,24.37,X40,ATH,IT
56,RAJESHEXPO,518.00,0.91,-63.27,172.45,0.07,86887.0,-86793.0,50384.0,1747.22,55.0,L-SC,2.31,268.0,-1.00,0.37,25.31,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,2.81,-12.29,14.04,0.02,21496.0,-21456.0,153103.0,133.26,53.0,M-SC,16.64,234.0,-1.00,1.14,33.17,OX40N,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.74
1,25,45.71
2,50,76.65


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.14
MC    30.33
LC    24.57
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.49
X40N     10.86
XR       10.10
XY25      9.37
AR        9.33
OX40N     7.81
SR        1.03
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.51
X-MC    23.56
X-LC    21.00
M-SC    12.19
X-SC     7.02
H-MC     5.00
L-SC     1.42
M-MC     1.41
M-LC     1.34
H-LC     1.18
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.36,-7.93,42.83
IT,12.76,-17.34,80.77
FINANCE,9.54,-20.80,71.84
ELECTRICAL,6.25,-11.82,52.78
PAINTS,6.11,-12.29,28.98
MISC,6.08,-58.92,124.02
INSURANCE,4.66,-1.01,36.61
PHARMA,4.11,-1.16,33.53
AUTO,3.39,-25.87,79.83


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3233537.0,21
AR,1318779.0,10
XR,1306094.0,13
X40,1014632.0,14
X40N,988063.0,9
OX40N,740786.0,10
XY25,381007.0,6
SR,282357.0,2
MH,83624.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3685137.0,25
M-SC,1422005.0,15
X-MC,1314867.0,16
X-SC,977012.0,8
X-LC,937653.0,11
H-MC,406800.0,3
L-SC,269843.0,3
M-LC,111780.0,1
H-LC,75402.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1288097.0      6
           AR         902563.0      5
M-SC       XY24       811612.0      6
H-SC       XR         782350.0      7
X-SC       X40N       505064.0      3
X-MC       X40        455003.0      7
           XY24       396927.0      3
X-LC       X40        381259.0      5
H-SC       OX40N      346146.0      4
M-SC       OX40N      307753.0      5
X-SC       XY24       293578.0      3
H-SC       SR         282357.0      2
X-MC       X40N       276379.0      4
X-LC       XY24       248980.0      2
H-MC       AR         212457.0      2
X-LC       X40N       206620.0      2
H-MC       XY24       194343.0      1
X-MC       XY25       186558.0      2
L-SC       XR         182956.0      2
X-SC       X40        178370.0      2
M-SC       XR         174283.0      2
           AR         128357.0      2
M-LC       XR         111780.0      1
X-LC       XY25       100794.0      2
L-SC       OX40N       86887.0      1
H-SC       MH          83624.0      1
H-LC       AR          75402.0      1
M-MC       XY25        55630.0      1
L-MC       XR          54725.0      1
L-LC       XY25        38025.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
